In [1]:
import torch

In [2]:
torch.tensor(1).view(1, 1, 1, 1)

tensor([[[[1]]]])

In [3]:
t1 = torch.tensor(
    [1., 2., 3.]
)
t1 = t1.view(1, 1, *t1.shape)

t2 = torch.tensor(
    [
        [1., 2., 3.],
        [4., 5., 6.]
    ]
)
t2 = t2.view(1, *t2.shape)

t3 = torch.randn(4, 4, 4)

In [4]:
t1

tensor([[[1., 2., 3.]]])

In [5]:
t2

tensor([[[1., 2., 3.],
         [4., 5., 6.]]])

In [6]:
t3

tensor([[[-0.4844,  0.6971, -2.5070,  0.7503],
         [ 0.7621,  0.4995,  0.2486, -1.6278],
         [-0.3657, -2.6311,  0.1609, -0.9479],
         [ 0.6166,  0.0069,  0.7965, -0.3574]],

        [[ 0.6118,  0.5416,  0.6147,  0.3276],
         [ 1.1116, -0.3271,  0.5995,  0.3316],
         [-0.7991, -1.3571,  0.5347,  0.2823],
         [ 0.4317, -2.5810, -0.0099,  1.0357]],

        [[ 0.7749,  1.3134, -2.1477, -0.9111],
         [-0.6290,  1.0245, -0.9009,  0.3782],
         [ 0.8140,  1.0206, -0.0617, -0.9668],
         [ 0.6249, -1.3853, -0.6815, -0.7274]],

        [[ 1.0435,  1.0950, -0.1458,  1.9956],
         [ 0.6967,  0.3744, -0.7028,  1.0507],
         [-2.5859,  0.5394,  0.8082,  2.0328],
         [ 0.5018,  0.3872, -0.4632, -1.1061]]])

In [7]:
t1.shape, t2.shape, t3.shape

(torch.Size([1, 1, 3]), torch.Size([1, 2, 3]), torch.Size([4, 4, 4]))

In [8]:
torch.stack([t1, t2, t3])

RuntimeError: stack expects each tensor to be equal size, but got [1, 1, 3] at entry 0 and [1, 2, 3] at entry 1

In [11]:
torch.cat([t1, t2], dim=2)

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 1 but got size 2 for tensor number 1 in the list.

In [ ]:
torch.cat((t1, t2, t3), dim=0)

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 1 but got size 2 for tensor number 1 in the list.

In [13]:
tensor1 = torch.randn(3, 3, 3)
tensor2 = torch.randn(4,4).view(1, 4, 4)

concat_tensor = torch.stack([tensor1, tensor2], dim=0)

RuntimeError: stack expects each tensor to be equal size, but got [3, 3, 3] at entry 0 and [1, 4, 4] at entry 1

In [14]:
tensor1 = torch.randn(3,3,3)
tensor2 = torch.randn(4,4)

tensor2_pad = torch.zeros(1,4,4)
tensor2_pad[0,:,:] = tensor2

concat_tensor = torch.cat([tensor1, tensor2_pad], dim=0)


RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 3 but got size 4 for tensor number 1 in the list.

In [15]:
t1 = torch.zeros(3, 3)
t2 = torch.tensor([[1, 2, 3], [1, 2, 3]])
t1[:t2.shape[0], :] = t2
t1

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [0., 0., 0.]])